## 定数

* Rubyは定数の値を変更できるという意味では変数とも言える
    * Warningは出る
    * 破壊的メソッドをつかって値を変更するときはWarningは出ない
* 命名規則
    * 先頭
        * 英数大文字
    * 構成文字
        * 英数字又はアンダースコア
* スコープ
    * 定数が定義されたクラス・モジュール
    * そのクラス・モジュール内で定義されたクラス・モジュール
    * そのクラス・モジュールをインクルードしているモジュール
    * クラス名やモジュール名で修飾すれば外部からアクセス可能
* 初期化されていない時の参照
    * 例外発生(NameError)
* <font color="red">メソッドの中で定義することはできない</font>
    * メソッドは複数回の実行が前提なので定数の初期化、更新が許されていない

In [1]:
### 変数名の先頭を大文字にすると定数になる
AAA = "hoge"
p AAA

### 警告は出るが再代入はできる
AAA = "fuga"
p AAA

### 破壊的メソッドによる変更は警告は出ない
AAA.concat("haga")
p AAA

### 初期化されていない時の参照は例外が発生
begin
  p BBB
rescue NameError => ex
  puts ex
end

"hoge"


(pry):6: warning: already initialized constant AAA
(pry):2: warning: previous definition of AAA was here


"fuga"
"fugahaga"
uninitialized constant BBB


In [2]:
### メソッドの中で定数を定義しようとするとエラーになる

def method1
  BBB = 1
end

SyntaxError: (eval):5: dynamic constant assignment
  BBB = 1
       ^


### ネストした定数

* モジュールオブジェクトやクラスオブジェクトが格納された定数に対しては「::」という演算子を指定してネストを指定できる

In [3]:
module ModuleA
end

### ModuleAの中に定数Aを宣言
ModuleA::A = 1

puts ModuleA::A

1


In [16]:
### モジュール式でネストした定数

module ModuleM
  A = 1
  class ClassB
    A = 2
  end
  class ClassC
    puts "ClassCには定数Aは無いので、ClassCの外側のModuleMの定数Aを参照する"
    puts "ClassC : A = #{A}"
  end
end

puts "ModuleMの定数AとClassBの定数Aは異なる"
puts ModuleM::A
puts ModuleM::ClassB::A

puts "ModuleMを再オープンして中に入って相対位置で定数を参照"
module ModuleM
  puts A
  puts ClassB::A
end

puts "ModuleMを再オープンして中に入ってルートからの絶対位置で定数を参照"
module ModuleM
  puts ::ModuleM::A
  puts ::ModuleM::ClassB::A
end


(pry):104: warning: already initialized constant ModuleM::A
(pry):29: warning: previous definition of A was here
(pry):106: warning: already initialized constant ModuleM::ClassB::A
(pry):31: warning: previous definition of A was here


ClassCには定数Aは無いので、ClassCの外側のModuleMの定数Aを参照する
ClassC : A = 1
ModuleMの定数AとClassBの定数Aは異なる
1
2
ModuleMを再オープンして中に入って相対位置で定数を参照
1
2
ModuleMを再オープンして中に入ってルートからの絶対位置で定数を参照
1
2


In [17]:
### ネストした現在のPATHの確認
module ModuleM
  class ClassC
    Module.nesting
  end
end

[ModuleM::ClassC, ModuleM]

### constantsメソッド

* Module#constants
    * Moduleクラスのインスタンスメソッド
    * 現在のスコープにある全ての定数を返す
    * 外部から参照可能な定数を調べる
    * レシーバ(またはそのスーパークラスやインクルードしているモジュール)に定義されている定数の名前を配列で返す
        * クラスは定数なのでクラス名も返す
* Module.constants
    * Moduleのクラスメソッド
    * 現在のプログラムのトップレベルにある全ての定数を返す

In [5]:
module ModuleN
  NNN = 1
  class ClassD
    DDD = 2
  end
end

puts ModuleN.constants
puts ModuleN::ClassD.constants

[:NNN, :ClassD]
[:DDD]


In [13]:
Module.constants

[:Object, :Module, :Class, :BasicObject, :Kernel, :NilClass, :NIL, :Data, :TrueClass, :TRUE, :FalseClass, :FALSE, :Encoding, :Comparable, :Enumerable, :String, :Symbol, :Exception, :SystemExit, :SignalException, :Interrupt, :StandardError, :TypeError, :ArgumentError, :IndexError, :KeyError, :RangeError, :ScriptError, :SyntaxError, :LoadError, :NotImplementedError, :NameError, :NoMethodError, :RuntimeError, :SecurityError, :NoMemoryError, :EncodingError, :SystemCallError, :Errno, :UncaughtThrowError, :ZeroDivisionError, :FloatDomainError, :Numeric, :Integer, :Fixnum, :Float, :Bignum, :Array, :Hash, :ENV, :Struct, :RegexpError, :Regexp, :MatchData, :Marshal, :Range, :IOError, :EOFError, :IO, :STDIN, :STDOUT, :STDERR, :ARGF, :FileTest, :File, :Dir, :Time, :Random, :Signal, :Proc, :LocalJumpError, :SystemStackError, :Method, :UnboundMethod, :Binding, :Math, :GC, :ObjectSpace, :Enumerator, :StopIteration, :RubyVM, :Thread, :TOPLEVEL_BINDING, :ThreadGroup, :ThreadError, :ClosedQueueError, :M

### const_missingメソッド

* const_missing
    * スーパークラスにも定数が見つからない場合に呼び出されるメソッド
    * このメソッドをサブクラスで上書きすれば定数が見つからない場合の動作を制御できる

In [6]:
### 定数が見つからないときメッセージを表示する
module ModuleO
  def self.const_missing(dummy)
    "Const not found"
  end
end

ModuleO::HOGE

"Const not found"

## ネームスペース

* 定数をまとめるだけのモジュール
    * 例えばRakeのTaskは他のクラスと名前が衝突することがよくあったので、Taskクラスの完全な名前は Rake::Task になった

In [ ]:
class Text   ### <- いかにもどこかに使われていて衝突しそうなクラス名
  
↓
  
module Bookworm
  class Text    ### 例えばBookworm::Textとすると、かぶる心配が少ない
    ...